# **Local Docker Testing**

| | |
|-|-|
| Author(s) | [Keeyana Jones](https://github.com/keeyanajones/) |

## **Overview**

Local Docker testing refers to the practice of building, running, and testing your application (or parts of it, like a microservice or an ML model) within Docker containers on your local development machine.  Its a fundamental part of a modern development workflow, especially in the context of microservices, cloud native applications, and MLOps.

### **Why is Local Docker Testing Important?**

1. Environment Consistency (Reproducibility):
   - **It works on my machine! Problem:** This is the primary problem Docker solves.  Docker containers package your application and all its dependencies into a single isolated unit.  Testing locally in Docker ensures that the environment your application runs in during development is identical to the production environment, reducing surprises during deployment.
   - **Eliminates Dependency Conflicts:** You don't have to worry about conflicting versions of libraries or runtimes installed on your host machine.  

2. Isolation:
   - Each container runs in its own isolated environment, preventing conflicts between different projects or services running on your machine. 
   - You can easily spin up and tear down multiple instances of your application or its dependencies (e.g., a database, a message queue) without cluttering your host system.
   
3. Faster Development Cycles:
   - **Quick Spin-up/Tear-down:** Containers start much faster than virtual machines, allowing for rapid iteration and testing.
   - **Simplified Dependency Management:** No need to manually install and configure complex services like databases; just define them in `docker-compose.yml`.

4. Simulating Production:
   - You can test how your application interacts with other services (databases, message queues, other microservices) by running them all as containers using tools like Docker Compose.  This closely mimics a production deployment.
   - Enables testing of networking, port mappings, and volume mounts before deploying to a more complex environment like Kubernetes.
   
5. Collaboration:
   - Developers on a team can share a `Dockerfile` and `docker-compose.yml` to ensure everyone is working with the same setup, leading to smother handoffs and fewer works on my machine issues.
   
6. CI/CD Alignment:
   - if your CI/CD pipeline uses Docker to build and test, local Docker testing helps catch issues earlier in the development cycle, reducing the feedback loop and ensuring successful CI/CD runs.

### **How to Perform Local Docker Testing:**

The process typically involves the following steps:

1. Create a Docker file:
   - Define the environment and steps to build your applications image.  This includes specifying a base image, copying code, installing dependencies, exposing ports, and defining the command to run your application.  (See previous explanation on Dockerfile creation).

2. Build the Docker image:
   - Use the `docker build` command to create an immutable image from your Dockerfile.   

In [ ]:
docker build -t my-app:latest .

3. Run the Docker Container:
   - Use the `docker run` command to create and start a container from you built image. You can map ports, mount volumes, set environment variables, etc.

In [ ]:
docker run -p 8000:8000 --name my-app-container my-app:latest

- You can then access your application (e.g., a web service) via `http://localhost:8000`

4. Use Docker Compose (for multiservice applications):
   - For applications composed of multiple services (e.g., a web app, a database, a redis cache).  Docker Compose is invaluable.  You define all services, their dependencies, and networking in a `docker-compose.yml` file.
   - `docker-compose.yml` example:

In [ ]:
version: '3.8'
services:
   web:
      build: .
      ports: 
         - "8000:8000"
      environment:
         DATABASE_URL: postgres://user:password@db:5432/mydb
      depends_on:
         - db
   db:
      image: postgres:13
      environment:
        POSTGRES_DB: mydb
        POSTGRES_USER: user
        POSTGRES_PASSWORD: password
      volumes:
         - db_data:/var/lib/postgresql/data
   volumes:
      db_data:

- Commands:

In [ ]:
docker-compose up -d # Build (if necessary) and start all services in detached mode
docker-compose logs  # View logs from all services 
docker-compose down  # Stop and remove all services and networks

5. Write and Run Tests:
   - **Unit Test:** Run unit tests inside the container during the build process (as a `RUN` instruction in the Dockerfile, often in a multi stage build) or by executing `docker exec` commands on a running container.
   - **Integration Test:** With Docker compose, you can spin up all dependent services and then run integration tests against your applications endpoint, ensuring inter service communication works as expected.
   - **End to End Tests:** Use tools like Selenium (for APIs) to interact with your containerized application as an end user would.

6. Debugging:
   - **Attaching to Containers:** Most IDEs and debuggers can attach to processes running inside a Docker container.   
   - **Logs:** `docker logs <container_name>` is essential for initial debugging. 
   - **Interactive Shell:** `docker exec -it <container_name> /bin/bash` (or `sh`) to get a shell inside the running container for inspection.   

### **Best Practices for Local Docker Testing:**

   - **Small, Focused Images:** Keep your Dockerfiles lean to make builds faster and images smaller.  Use multi-stage builds to separate build dependencies from runtime dependencies.
   - **Leverage Build Cache:** Order your Dockerfile instructions form least to most frequently changing to maximize cache hits during `docker build`. 
   - **Use `.dockerignore`:** Exclude unnecessary files (e.g., `.git`, `node_modules`, `__pycache__`, `venv`) from the build context to speed up builds and reduce image size. 
   - **Environment Variables:** Use environment variables for configuration (e.g., database URLs, API keys) instead of hardcoding them in the Dockerfile.
   - **Health Checks:** Define `HEALTHCHECK` instructions in your Dockerfile or Docker Compose to ensure containers are actually ready to serve requests before traffic is directed to them.
   - **Volume Mounting for Development:** For rapid iteration on code changes, mount your local source code directory into the container using a Docker volume (`-v /path/to/local/code:/app`). This allows you to modify code on you host and see changes reflected in the container without rebuilding the image every time (requires live reloading in your application).
   - **Container Naming:** Use meaningful names for your containers (`--name`) to easily identify and manage them.
   - **Clean Up:** Regularly remove stopped containers (`docker rm`) and unused images (`docker rmi`) to free up disk space.  Use `docker system prune` for a comprehensive cleanup.
   - **Adopt Docker Compose Early:** Even for single service apps, if you anticipate adding a database or another service, starting with Docker Compose can simplify future expansion.

By consistently using Docker for local development and testing, you significantly streamline your workflow, improve collaboration, and build more reliable applications that seamlessly transition from your laptop to production.

----